**Imports**

In [ ]:
import os
import warnings
import tensorflow as tf
tf.get_logger().setLevel('INFO')

# Suppress TensorFlow's native logging (INFO and WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Suppress Python warnings including deprecation and future warnings from TensorFlow
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", module='tensorflow')

In [ ]:
import importlib
import preprocess 
import train
import evaluate
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def reload_all():
    importlib.reload(preprocess)
    importlib.reload(train)
    importlib.reload(evaluate)
    print("Modules reloaded.")

# Then just run:
reload_all()

Modules reloaded.


**Model Configuaration**

In [4]:
model_names = ["ResNet50", "VGG16", "EfficientNetB0", "InceptionV3"]

**Preprocess Data**

In [5]:
X_train, X_test, y_train, y_test = preprocess.preprocess_data()

Data preprocessing and augmentation completed.


**Results Directory**

In [6]:
results = {}

**Train and Evaluate**

In [ ]:
# Loop through each model
for model_name in model_names:
    print(f"Training and evaluating model: {model_name}")
    
    # Train Model
    trained_model, avg_roc_auc = train.train_k_fold(X_train, y_train, model_name)
    
    # Store the average ROC AUC in results
    results[model_name] = {
        'model': trained_model,
        'average_roc_auc': avg_roc_auc
    }
    
    # Save the trained model to disk for evaluation
    
    SAVE_DIR = "saved_models"
    os.makedirs(SAVE_DIR, exist_ok=True)
    trained_model.save(os.path.join(SAVE_DIR, f"final_trained_{model_name}.keras"))

    # Evaluate Model using the function from evaluate.py
    print(f"Evaluating model: {model_name}")
    evaluate.evaluate_model(model_name, X_test, y_test)

Training and evaluating model: ResNet50


Training fold 1/5 for ResNet50
Epoch 1/10


58/58 [==============================] - ETA: 0s - loss: 1.3267 - accuracy: 0.3503

C:\Users\Chris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


58/58 [==============================] - 33s 539ms/step - loss: 1.3267 - accuracy: 0.3503 - val_loss: 1.2972 - val_accuracy: 0.3042
Epoch 2/10
58/58 [==============================] - ETA: 0s - loss: 1.2600 - accuracy: 0.4215

**Sample Image Predictions**

In [3]:
# Binarize the true labels
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Calculate the ROC curve
fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_pred.ravel())
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

plt.figure(figsize=(12, 4))

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Training and Validation Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'LabelBinarizer' is not defined

**Display Results**

In [ ]:
for model_name, result in results.items():
    print(f"Model: {model_name}, Average ROC AUC: {result['average_roc_auc']}")